# Annotating genes for obtaining PPI network

This notebook is used for annotating the genes with their corresponding IDs for constructing the PPI network.

## Libraries

In [2]:
import pandas as pd
import numpy as np
import mygene
import requests
import io
import shutil

## Loading data

Arbitrary selection, we could later select all the files in the folder for annotating every file with its gene ID.

In [3]:
project_name = "PRJNA248469"
data = pd.read_csv(f'data/IBD_Andre_data/{project_name}_matrix.tsv', sep='\t', index_col = "Geneid")
data

,SAMN03322967,SAMN03322968,SAMN03322969,SAMN03322970,SAMN03322971,SAMN03322972,SAMN03322973,SAMN03322974,SAMN03322975,SAMN03322976,...,SAMN03323287,SAMN03323288,SAMN03323290,SAMN03323292,SAMN03323293,SAMN03323294,SAMN03323295,SAMN03323296,SAMN03323297,SAMN03323298
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000284662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000186827,120,53,146,144,56,74,34,178,101,30,...,95,51,27,31,89,15,86,63,47,215
ENSG00000186891,36,53,111,106,30,93,49,107,57,38,...,54,59,32,43,74,18,70,55,50,135
ENSG00000160072,223,173,157,223,144,151,53,222,93,269,...,174,289,211,187,308,181,226,132,274,349
ENSG00000041988,135,137,122,133,74,78,42,163,67,131,...,108,109,92,79,115,82,108,66,128,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000271254,81,52,12,46,46,35,17,7,11,95,...,16,40,147,30,52,71,26,21,61,8
ENSG00000275987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000268674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Mapping gene IDs

We start with getting the Ensembl version numbers and map Ensembl IDs -> gene. We need to take into account the other molecules that can be in our data (e.g. RNAs, pseudo-genes, non-coding genes, etc.) so we put a filter to only extract protein-coding genes.

In [4]:
gene_data = data.copy()

ensembl_ids = gene_data.index.tolist()
# Sanity check: keep only gene IDs
ensembl_ids = [i for i in ensembl_ids if str(i).upper().startswith("ENSG")]

mg = mygene.MyGeneInfo()

fields = "symbol, type_of_gene, entrezgene, ensembl"
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields=fields, species='human', as_dataframe=True)
gene_info = gene_info.loc[~gene_info.index.duplicated(keep='first')]

# Find column that have gene-type/biotype information
type_cols = [c for c in gene_info.columns if "type" in c.lower() or "biotype" in c.lower()]

if type_cols:
    tcol = type_cols[0]
    is_protein = gene_info[tcol].astype(str).str.lower().str.contains("protein")
    gene_info = gene_info[is_protein]

gene_data.insert(0, "gene_symbol", gene_info["symbol"].reindex(gene_data.index))

gene_data = gene_data.dropna(subset=["gene_symbol"]).set_index("gene_symbol")

print(f"After mapping from {data.shape[0]} genes only {gene_data.shape[0]} remain.")

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
35 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
1209 input query terms found no hit:	['ENSG00000272482', 'ENSG00000224621', 'ENSG00000234166', 'ENSG00000225643', 'ENSG00000287400', 'ENS


After mapping from 60664 genes only 19465 remain.


Now we start filtering low count (threshold set to 1 CPM in at least 10% of samples) and low variance genes.

In [5]:
def filter_by_cpm(counts_df, threshold = 1, min_frac = 0.1):
    # Calculate total counts for each gene
    lib_size = counts_df.sum(axis=0)
    # Calculate CPM
    cpm = counts_df.div(lib_size, axis=1) * 1e6
    # Filter genes below threshold
    keep = (cpm >= threshold).sum(axis=1) >= np.ceil(min_frac * counts_df.shape[1])
    return counts_df.loc[keep, :], keep

def filter_by_var(counts_df, pct = 0.5):
    # Normalize counts by library size (CPM)
    lib_size = counts_df.sum(axis=0)
    cpm = counts_df.div(lib_size, axis=1) * 1e6
    logcpm = np.log2(cpm + 1.0)
    # Calculate variance for each gene across samples
    gene_var = logcpm.var(axis=1)
    # logcpm percentile - keep top pct counts
    q = 1.0 - pct
    cutoff = gene_var.quantile(q)
    keep = gene_var >= cutoff
    return counts_df.loc[keep, :], keep

data_cpm_filtered, _ = filter_by_cpm(gene_data, threshold=1, min_frac=0.1)
data_filtered, _ = filter_by_var(data_cpm_filtered, pct=0.1)

print(f"Filtered from {gene_data.shape[0]} to {data_filtered.shape[0]} genes.")

Filtered from 19465 to 1511 genes.


## Extracting PPI network from STRING

With the genes we have obtained now we are going to retrieve the PPI network corresponding for those from STRING. At the end we are going to retain genes that appear in STRING DB.

In [6]:
gene_list = data_filtered.index.tolist()

# gene_list = ["TP53", "BRCA1", "EGFR", "MYC", "PTEN"]

species_id = 9606 # Homo sapiens = 9606

confidence_score = 600 # confidence score threshold - 900 is highest confidence!

# Base URL
string_api_url = "https://string-db.org/api"

# API method to get network
method = "network"

# Format list of genes into single string
id_string = "\n".join(gene_list)

# Request URL construct
request_url = "/".join([string_api_url, "tsv", method])

# Parameters for the API call
params = {
    "identifiers": id_string,
    "species": species_id,
    "required_score": confidence_score,
    "caller_identity": "script"
}

# Making API call
try:
    response = requests.post(request_url, data=params)
    response.raise_for_status()

    # io.StringIO function treats the response text as file
    ppi_network = pd.read_csv(io.StringIO(response.text), sep="\t")

    print("Successfully retrieved PPI network!")
    print(f"Found {len(ppi_network)} interactions.")
    print(ppi_network.head())

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")

except Exception as err:
    print(f"An error occurred: {err}")

Successfully retrieved PPI network!
Found 7829 interactions.
             stringId_A            stringId_B preferredName_A preferredName_B  \
0  9606.ENSP00000001146  9606.ENSP00000456609         CYP26B1           STRA6   
1  9606.ENSP00000001146  9606.ENSP00000310721         CYP26B1          CYP7B1   
2  9606.ENSP00000001146  9606.ENSP00000337224         CYP26B1            LRAT   
3  9606.ENSP00000001146  9606.ENSP00000320401         CYP26B1         UGT2B17   
4  9606.ENSP00000001146  9606.ENSP00000251566         CYP26B1          UGT2A3   

   ncbiTaxonId  score  nscore  fscore  pscore  ascore  escore  dscore  tscore  
0         9606  0.654     0.0     0.0   0.000   0.045   0.054    0.00   0.647  
1         9606  0.660     0.0     0.0   0.457   0.051   0.049    0.00   0.388  
2         9606  0.681     0.0     0.0   0.000   0.086   0.000    0.00   0.665  
3         9606  0.687     0.0     0.0   0.000   0.067   0.000    0.67   0.065  
4         9606  0.687     0.0     0.0   0.000   0.06

Finally, we retain gene counts that appear in the PPI network.

In [7]:
ppi_gene_symbols = set(ppi_network["preferredName_A"]).union(ppi_network["preferredName_B"])
gene_data_in_ppi = gene_data[gene_data.index.isin(ppi_gene_symbols)]

print(f"From {data_filtered.shape[0]} genes, only {gene_data_in_ppi.shape[0]} appear in PPI network.")

From 1511 genes, only 1165 appear in PPI network.


## Save to file

Now with the PPI network and the filtered gene expression data we processed to save all this information.

In [8]:
# Save gene_data_in_ppi with gene symbols as a column (not as index)
gene_data_in_ppi.reset_index().to_csv(f"data/IBD_Andre_data_filtered/{project_name}_matrix_filtered.tsv", sep="\t", index=False)
ppi_network.to_csv(f"data/IBD_Andre_data_filtered/{project_name}_ppi_network.tsv", sep="\t", index=False)

# optional but useful, copy metadata to new folder
src_metadata = f"data/IBD_Andre_data/{project_name}_metadata.txt"
dst_metadata = f"data/IBD_Andre_data_filtered/{project_name}_metadata.txt"
shutil.copy(src_metadata, dst_metadata)

'data/IBD_Andre_data_filtered/PRJNA248469_metadata.txt'